### init the engine

In [1]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://mahtin@localhost:5432/mahtin', echo=False)

### declare a schema

In [2]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [3]:
from sqlalchemy import Column, Integer, String
class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(80))
    fullname = Column(String(80))
    nickname = Column(String(50))
    
    def __repr__(self):
        return f"<User(name={self.name}, fullname={self.fullname}, nickname={self.nickname})"

### create the table(s)

In [4]:
Base.metadata.create_all(engine)

### creating a session class

In [5]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine) # the bind can be defined later over Session.configure(bind=engine)
session = Session()

### initiate a session and add objects

In [6]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='Eddy')
ed_user.fullname
session.add(ed_user) # session is pending
our_user = session.query(User).filter_by(name='ed').first()
our_user

'Ed Jones'

<User(name=ed, fullname=Ed Jones, nickname=Eddy)

In [7]:
session.add_all([
...     User(name='wendy', fullname='Wendy Williams', nickname='windy'),
...     User(name='mary', fullname='Mary Contrary', nickname='mary'),
...     User(name='fred', fullname='Fred Flintstone', nickname='freddy')])

In [8]:
ed_user.nickname = 'Eddo'

In [9]:
session.commit()

## Querying

+ query returns named tuples (after attributes or class in query) for every query object

In [10]:
for tuple in session.query(User, User.fullname).order_by(User.id):
    print(tuple.User.id, tuple.fullname)

1 Ed Jones
2 Wendy Williams
3 Mary Contrary
4 Fred Flintstone


+ label your resulting tuple name using label and aliased

In [11]:
from sqlalchemy.orm import aliased
alUser = aliased(User, name='alUser')
for tuple in session.query(alUser, User.name.label('the_name'))[1:3]:
    print(tuple.alUser.name, tuple.the_name)

ed wendy
ed mary


### filter and filter_by
* filter_by gives simple attribute identities
* filter allows complex expressions
* all filter commands are fully chainable

In [12]:
session.query(User).order_by(User.id).filter_by(fullname='Ed Jones').first()

<User(name=ed, fullname=Ed Jones, nickname=Eddo)

In [13]:
session.query(User).order_by(User.id).filter(User.fullname == 'Ed Jones').first()

<User(name=ed, fullname=Ed Jones, nickname=Eddo)

In [14]:
session.query(User).filter_by(fullname='Ed Jones').filter(User.name == 'ed').first()

<User(name=ed, fullname=Ed Jones, nickname=Eddo)

In [15]:
session.query(User).filter(User.fullname.like('%Ed%')).first()

<User(name=ed, fullname=Ed Jones, nickname=Eddo)

In [16]:
session.query(User).filter(User.name.in_(['ed', 'wendy'])).all()

[<User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=wendy, fullname=Wendy Williams, nickname=windy)]

In [17]:
from sqlalchemy import and_, or_
session.query(User).filter(or_(User.name.in_(['ed', 'wendy']), User.name == 'fred')).all()

[<User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=wendy, fullname=Wendy Williams, nickname=windy),
 <User(name=fred, fullname=Fred Flintstone, nickname=freddy)]

+ you can use full SQL statements with from_statement()

In [18]:
from sqlalchemy import text
session.query(User).from_statement(text("SELECT * FROM users WHERE name=:value").params(value='ed')).first()

<User(name=ed, fullname=Ed Jones, nickname=Eddo)

### Relationships

In [8]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship
class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship("User", back_populates='addresses')
    
    def __repr__(self):
        return f"<Address(email_address={self.email_address})>"

# this is resolved as a collection (configurable but defaulting to a list)
User.addresses = relationship("Address", order_by=Address.id, back_populates="user")


In [20]:
Base.metadata.create_all(engine)

In [21]:
jack = User(name='jack', fullname='Jack Bean', nickname='gjffdd')

In [22]:
jack.addresses = [
...                 Address(email_address='jack@google.com'),
...                 Address(email_address='j25@yahoo.com')]

In [23]:
jack.addresses

[<Address(email_address=jack@google.com)>,
 <Address(email_address=j25@yahoo.com)>]

In [24]:
session.add(jack)
session.commit()

In [25]:
jack.addresses

[<Address(email_address=jack@google.com)>,
 <Address(email_address=j25@yahoo.com)>]

In [26]:
session.query(Address).all()

[<Address(email_address=jack@google.com)>,
 <Address(email_address=j25@yahoo.com)>]

### Joins

In [27]:
for u, a in session.query(User, Address).\
    filter(User.id==Address.user_id).\
    filter(Address.email_address=='jack@google.com').\
    all():
        print(u)
        print(a)

<User(name=jack, fullname=Jack Bean, nickname=gjffdd)
<Address(email_address=jack@google.com)>


As a default, a `join()` joins two dbs on their Foreign keys

In [28]:
session.query(User).join(Address).\
    filter(Address.email_address=='jack@google.com').\
    all()

[<User(name=jack, fullname=Jack Bean, nickname=gjffdd)]

Else, you can use more explicit join statements:

In [29]:
session.query(User).join(Address, User.id==Address.user_id).all()

[<User(name=jack, fullname=Jack Bean, nickname=gjffdd)]

In [30]:
session.query(User).join(User.addresses).all()

session.query(User).join(Address, User.addresses).all()

session.query(User).join('addresses').all()

[<User(name=jack, fullname=Jack Bean, nickname=gjffdd)]

[<User(name=jack, fullname=Jack Bean, nickname=gjffdd)]

[<User(name=jack, fullname=Jack Bean, nickname=gjffdd)]

+ join only retrieves Users, where address field is defined
+ for all Users, you can use outerjoin

In [31]:
session.query(User).outerjoin(User.addresses).all()

[<User(name=jack, fullname=Jack Bean, nickname=gjffdd),
 <User(name=wendy, fullname=Wendy Williams, nickname=windy),
 <User(name=fred, fullname=Fred Flintstone, nickname=freddy),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=mary, fullname=Mary Contrary, nickname=mary)]

In [32]:
session.query(User, Address).select_from(Address).join(User).all()

[(<User(name=jack, fullname=Jack Bean, nickname=gjffdd),
  <Address(email_address=jack@google.com)>),
 (<User(name=jack, fullname=Jack Bean, nickname=gjffdd),
  <Address(email_address=j25@yahoo.com)>)]

### Aliases
+ for several references to same table you can use aliased()
+ then you join the different aliases of same table to retrieve several values

In [33]:
from sqlalchemy.orm import aliased
add_alias1 = aliased(Address)
add_alias2 = aliased(Address)
session.query(User.name, add_alias1.email_address, add_alias2.email_address).\
    join(add_alias1, User.addresses).\
    join(add_alias2, User.addresses).\
    filter(add_alias1.email_address == 'jack@google.com').\
    filter(add_alias2.email_address == 'j25@yahoo.com').all()


[('jack', 'jack@google.com', 'j25@yahoo.com')]

### subqueries
+ first create a subquery using .subquery() which returns an SQL expression construct
+ here, we use subquery for an aggregate function count()
+ the subquery can be treated like a standard table with attributes accessible as c.

In [1]:
from sqlalchemy import func
sub_query = session.query(Address.user_id, func.count('*').label('address_count')).\
    group_by(Address.user_id).subquery()
sub_query.c.address_count

NameError: name 'session' is not defined

In [35]:
session.query(User, sub_query.c.address_count).\
    outerjoin(sub_query, User.id == sub_query.c.user_id).\
    order_by(User.id).all()

[(<User(name=ed, fullname=Ed Jones, nickname=Eddo), None),
 (<User(name=wendy, fullname=Wendy Williams, nickname=windy), None),
 (<User(name=mary, fullname=Mary Contrary, nickname=mary), None),
 (<User(name=fred, fullname=Fred Flintstone, nickname=freddy), None),
 (<User(name=jack, fullname=Jack Bean, nickname=gjffdd), 2)]

### Exists

In [36]:
from sqlalchemy.sql import exists
existing = exists().where(Address.user_id != User.id)
session.query(User.name).filter(existing).all()

[('ed'), ('wendy'), ('mary'), ('fred')]

+ EXISTS can be expresses as any(condition) in SQLalchemy

In [37]:
session.query(User.name).filter(~User.addresses.any()).all()

[('ed'), ('wendy'), ('mary'), ('fred')]

In [38]:
session.query(User.name).filter(User.addresses.any(Address.email_address.like('%google%'))).all()

[('jack')]

### Eager Loading
+ if you want to load associated data in a single transaction, you can use selectinload
+ in selectinload, the related data is loaded in a separate select call

In [39]:
from sqlalchemy.orm import selectinload
session.query(User).options(selectinload(User.addresses)).\
    filter_by(name='jack').first().addresses

[<Address(email_address=jack@google.com)>,
 <Address(email_address=j25@yahoo.com)>]

+ in joinedload, the data is joined in one select call

In [40]:
from sqlalchemy.orm import joinedload
session.query(User).options(joinedload(User.addresses)).\
    filter_by(name='jack').first().addresses

[<Address(email_address=jack@google.com)>,
 <Address(email_address=j25@yahoo.com)>]

+ contains_eager preloads the associated data to be used in chained filters
+ in example, Address.user is eager_loaded to allow filtering with User.name

In [41]:
from sqlalchemy.orm import contains_eager
session.query(Address).join(Address.user).\
    filter(User.name=='jack').\
    options(contains_eager(Address.user)).all()

[<Address(email_address=jack@google.com)>,
 <Address(email_address=j25@yahoo.com)>]

### Deleting
+ without cascading, data related to the deleted data will still be available

In [42]:
session.delete(jack)
session.query(Address).filter(Address.email_address.in_(['jack@google.com', 'j25@yahoo.com'])).count()

2

+ with cascading, linked data is deleted as well
+ here, we reset and declare the User class with cascading

In [45]:
session.close()
Base = declarative_base()

class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
     
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)
    addresses = relationship("Address", back_populates='user',
        cascade="all, delete, delete-orphan")

    def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (
            self.name, self.fullname, self.nickname)
    
    
class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address
    
Base.metadata.create_all(engine)


+ with cascading activate, all associated data will be removed with the object

In [46]:
session.delete(jack)
session.query(Address).filter(Address.email_address.in_(['jack@google.com', 'j25@yahoo.com'])).count()

2

### Many to Many Relationship
+ for many2many, you can use an association table to combine both related tables

In [9]:
from sqlalchemy import Table, Text
# association table
post_keywords = Table('post_keywords', Base.metadata,\
            Column('post_id', ForeignKey('posts.id'), primary_key=True),\
            Column('keyword_id', ForeignKey('keywords.id'), primary_key=True))

In [10]:
class BlogPost(Base):
    __tablename__ = 'posts'
    
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.id'))
    headline = Column(String(255), nullable=False)
    body = Column(Text)
    
    # here the relationship via the association table is created
    keywords = relationship('Keyword', secondary=post_keywords, back_populates='posts')
    
    # init is optional in SQLalchemy
    def __init__(self, headline, body, author):
        self.author = author
        self.headline = headline
        self.body = body
        
    def __repr__(self):
        return f"BlogPost({self.headline}, {self.body}, {self.author})"
    
class Keyword(Base):
    __tablename__ = 'keywords'
    
    id = Column(Integer, primary_key=True)
    keyword = Column(String(50), nullable=False, unique=True)
    posts = relationship('BlogPost', secondary=post_keywords, back_populates='keywords')
    
    # init is optional in SQLalchemy
    def __init__(self, keyword):
        self.keyword = keyword
    

In [11]:
BlogPost.author = relationship(User, back_populates='posts')
User.posts = relationship(BlogPost, back_populates="author", lazy="dynamic")

In [12]:
Base.metadata.create_all(engine)

In [ ]:
wendy = session.query(dUser).filter_by(name='wendy').one()
post = BlogPost("Wendy's Blog Post", "This is a test", wendy)
session.add(post)

In [ ]:
post.keywords.append(Keyword('wendy'))
post.keywords.append(Keyword('firstpost'))

In [ ]:
session.commit()

In [ ]:
session.query(BlogPost).filter(BlogPost.keywords.any(keyword='firstpost')).all()

In [ ]:
session.query(BlogPost).filter(BlogPost.author==wendy).\
filter(BlogPost.keywords.any(keyword='firstpost')).all()

# Pandas with SQLAlchemy

In [ ]:
data = pd.read_sql_table('users', engine)

In [ ]:
data

In [ ]:
for d in data:
    print(d)